#### Agenda - Given a sequence of previous characters, model the probability distribution of the next character in the sequence. 

Here we try harry potter text

We will be using kears for this note 

In [1]:
from __future__ import print_function

import numpy as np
import random
import sys

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

Using TensorFlow backend.


# Get data

In [3]:
# load data
path = "./../data/harry_potter_3.txt"
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 611606


In [4]:
total_chars = sorted(list(set(text)))
print('total chars:', len(total_chars))


total chars: 53


In [5]:
#dictionaries to map characters to IDs and vice-a-versa

char_indices = dict((c, i) for i, c in enumerate(total_chars))
indices_char = dict((i, c) for i, c in enumerate(total_chars))


In [6]:
# cut the text in semi-redundant sequences of maxlen characters

maxlen = 40 # length of data window
step = 1    # step by which to shift the data window 

sentences = []    # this stores X
next_chars = []   # this stores y

# (i:i+40) as X, (i+40) as corresponding y
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen]) # get sentence
    next_chars.append(text[i + maxlen])   # get next character
    

num_of_sentences = len(sentences)

print('Number of datapoints/sequences:', num_of_sentences)

Number of datapoints/sequences: 611566


In [7]:
#Vectorize the input 

# X will be a 3D tensor - num_of_sentences * maxlen * total_chars
# y will be 2D tensor - sentences * total_chars

print('Vectorization...')
X = np.zeros((num_of_sentences, maxlen, len(total_chars)), dtype=np.bool)
y = np.zeros((num_of_sentences, len(total_chars)), dtype=np.bool)

# populate the tensors
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


# Model

In [ ]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()

# lstm takes a single slice of 3D tensor along num_of_sentences axiss
model.add(LSTM(128, input_shape=(maxlen, len(total_chars)))) 

# add a dense layer - takes 128 input and spits output of num_of_chars 
model.add(Dense(len(total_chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
#Preds is scores over output space (characters)
# Its nothing but - confidence the RNN currently assigns to each character coming next in the sequence

# we convert scores into probability distribution and pick the best bet. 

# Temperature. We can also play with the temperature of the Softmax during sampling. Decreasing the temperature 
# from 1 to some lower number (e.g. 0.5) makes the RNN more confident, but also more conservative in its samples. 
# Conversely, higher temperatures will give more diversity but at cost of more mistakes (e.g. spelling mistakes, 
# etc). In particular, setting temperature very near zero will give the most likely thing

def sample(preds, temperature=1.0):
    
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds) # convert scores to prob via softmax
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas) # return the one with max probability

In [10]:
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,batch_size=128,epochs=1)
    
    start_index = random.randint(0, len(text) - maxlen - 1) # pick an index at random
    
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)
        
        generated = ''
        sentence = text[start_index: start_index + maxlen] # pick sentence at randomly gerenated index
        generated += sentence
        
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        sys.stdout.write("\n")
        
        for i in range(400):
            x = np.zeros((1, maxlen, len(total_chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1. # put the sentence in 1-hot format
                
            preds = model.predict(x, verbose=0)[0] # make the model spit out predictions
            
            next_index = sample(preds, diversity) # choose a character index
            next_char = indices_char[next_index]  # get the corresponding chracater
            
            generated += next_char                # append this character to the sentence
            sentence = sentence[1:] + next_char   # now shift the the sentence by one character
            
            sys.stdout.write(next_char)           # write the character to buffer
            sys.stdout.flush()
            
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
611566/611566 [==============================] - 810s 1ms/step - loss: 2.1640

----- diversity: 0.2
----- Generating with seed: "pine,
and the book shivered, and then fe"
pine,
and the book shivered, and then fe
et streak and she stright and the sure the she came and she was she harry, and she still the she streambeath the she con she streanst as the grimbed and she she the said and harry she sure the she care the gase the got in the gone the she could as the she carned the right and the give as his she said harry sure harry and as the she they she seeped the side and he starned the grase as the ginule th

----- diversity: 0.5
----- Generating with seed: "pine,
and the book shivered, and then fe"
pine,
and the book shivered, and then fe
et he wase got sevel the had the give get bus the shough the masick, in them the fere the sumpel and the stanter up as the gace the severing the snape where out then with then his

/home/piyush/Piyush/study/anthill/anthill/lib/python3.5/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app


e croum his his his his his him.

"i chen his hermione harry was his his his here harry was his his his his harry cround hermione could his harry seen the caster his his hermione harry was hermione sumfref with the classide and hermione scabbers of his his hermione with the cast his hermione and hermione his harry could his his his him.

"hermione seen 

----- diversity: 0.5
----- Generating with seed: ""very smart owl you've got there, chuckl"
"very smart owl you've got there, chuckl
e. the farge his but they the crook.

"hermione hermiously. "i have with the sit were his was the seemed on his his his see a dement was his his his here harry he had the
was only, still his scabbers is with his with his his screak it. he son't cup in the bearly.

"she wey with ammisging some streaked his his his me the having his a were come a had streaming atter the cup; he speen the back.

"we'

----- diversity: 1.0
----- Generating with seed: ""very smart owl you've got there, chuckl"
"very smart owl 

them die. thinklelv? harry burn ristle. momar and himing the hand me knower
af groad, wood. "i last. o wiple stureded how ey, carew-pre you sumorted... he will pully, hiddid, oly herry 

----- diversity: 1.2
----- Generating with seed: "
"excellent, harry," lupin muttered as h"

"excellent, harry," lupin muttered as h
all, she jcops -- head and possince worder owided permiped a many
magictoen, it, hope had tibviksest hor ord. aid one more. it soot i at mardy. gay swoedued thincy fully throut whe sucles, harry
look take
poote grown't looking a legry croakeghowed hime! a heem on dan them, loud with aloar dot
gavew-whaurderabreean!
ole croiret watteduadin dunthed olized sidgy
and it knor nericulir, and harry i azk

--------------------------------------------------
Iteration 6
Epoch 1/1
611566/611566 [==============================] - 435s 711us/step - loss: 3.2656

----- diversity: 0.2
----- Generating with seed: "inistry providing cars, father?" percy a"
inistry providing cars, father?"

KeyboardInterrupt: 